## 导入需要的库

In [ ]:
import os
import cv2
import logging
import tensorflow as tf
import face_recognition
import math
import numpy as np
import dlib
import PythonMagick
import matplotlib.pyplot as plt
from scipy import linalg,ndimage 
from numpy import array,zeros
from PIL import Image 

## 变换计算过程

In [ ]:
def compute_rigid_transform(rp,p):
    A=array([[p[0],-p[1],1,0],[p[1],-p[0],0,1],
             [p[2],-p[3],1,0],[p[3],-p[2],0,1],
             [p[4],-p[5],1,0],[p[5],-p[4],0,1],
             [p[6],-p[7],1,0],[p[7],-p[6],0,1],
             [p[8],-p[9],1,0],[p[9],-p[8],0,1]])
    y=array([rp[0],rp[1],rp[2],rp[3],rp[4],rp[5],rp[6],rp[7],rp[8],rp[9]])
    a, b, tx, ty=linalg.lstsq(A,y)[0]
    R=array([[a,-b],[b,a]])
    return R,tx,ty

## 计算人脸关键点

In [ ]:
def calc_point(img):
    predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
    detector = dlib.get_frontal_face_detector() 
    dets = detector(img, 1)
    shape=[]
    for i, d in enumerate(dets):
         shape = predictor(np.uint8(img), d)
    #人脸检测工具 可检测人脸中68个关键点
    return shape

## 定义人脸旋转函数

In [ ]:
def rotate(img):
    shape=calc_point(img)#检测人脸关键点
    eye_center=((shape.part(36).x+shape.part(45).x)*1./2,(shape.part(36).y+shape.part(45).y)*1./2) # 计算两眼的中心坐标
    dx=(shape.part(45).x-shape.part(36).x) 
    dy=(shape.part(45).y-shape.part(36).y)
    angle=math.atan2(dy,dx)*180./math.pi # 计算角度
    RotateMatrix=cv2.getRotationMatrix2D(eye_center,angle,scale=1) # 计算仿射矩阵
    img=cv2.warpAffine(img, RotateMatrix,(img.shape[1], img.shape[0])) # 进行放射变换，即旋转
    return img

## 总体调用函数

In [ ]:
def move(img):
    shape = calc_point(img)#检测人脸关键点
    order = [36, 45, 30, 48, 54]
    #左眼 右眼 鼻子 左嘴角 右嘴角 所对应的坐标在关键点序列中的位置
    p=[]
    rp=[75,125,125,125,100,160,80,180,125,180]
    #要对齐的坐标序列
    for j in order:#构造向量
        x = shape.part(j).x
        y = shape.part(j).y
        p.append(x)
        p.append(y)
    R,tx,ty=compute_rigid_transform(rp,p)
    T=array([[R[1][1],R[1][0]],[R[0][1],R[0][0]]]) # 构造变换矩阵T
    aff_img=zeros(img.shape,"uint8")
    for i in range(len(img.shape)):
        aff_img[:,:,i]=ndimage.affine_transform(img[:,:,i],linalg.inv(T),offset=[-ty,-tx])
    #进行仿射变换
    return aff_img

## 对五张图片依次对齐

In [ ]:
num=5
for i in range(1,num+1):
    filename=str(i)+".jpg"
    img = PythonMagick.Image(filename)
    img.sample('200x250!')
    img.write(filename)
    #对图片做预处理 调整为合适的大小
    face_alignment(filename)
    #人脸对齐操作